# TP1 - Part 3: Will Your Drug Reach the Brain?

**Day 2 - AI for Sciences Summer School**

**Instructor:** Raphael Cousin

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/racousin/ai_for_sciences/blob/main/day2/tp1_part3.ipynb)

---

## The Scenario

You are a computational chemist at a pharmaceutical company working on **Alzheimer's disease**. Your team has designed a promising new molecule called **AZD-2024** that might inhibit the formation of amyloid plaques in the brain.

But before spending months on synthesis and animal testing, your manager asks:

> *"Can this molecule even reach the brain? If it can't cross the blood-brain barrier, the whole project is dead before it starts."*

You decide to build an AI model to predict BBB permeability **before** any wet lab experiment.

---

## The Blood-Brain Barrier Challenge

The **blood-brain barrier (BBB)** is a highly selective membrane that protects the brain from pathogens and toxins. Unfortunately, it also blocks ~98% of small-molecule drugs.

```
                          BLOOD VESSEL
    ════════════════════════════════════════════════
         Drug molecules floating in bloodstream
              💊  💊  💊  💊  💊  💊
    ────────────────────────────────────────────────
         BLOOD-BRAIN BARRIER (tight junctions)
    ════════════════════════════════════════════════
              💊              💊       ← Only some get through!
                    BRAIN TISSUE
    ════════════════════════════════════════════════
```

For Alzheimer's drugs, BBB permeability is **essential** - the drug MUST reach the brain to work.

---

## What You'll Learn

1. How to use **pre-trained embeddings as features** for classification
2. How to build a classifier **starting simple**, then improving
3. How to **evaluate** a classifier beyond accuracy
4. How to **apply your model** to predict new molecules

---

# Part 1: The Strategy

## Transfer Learning with Embeddings

We don't have millions of BBB-labeled molecules to train a model from scratch. But we CAN:

1. Use **ChemBERTa** (trained on 77M molecules) to convert SMILES → embeddings
2. Train a **classifier** on top of these embeddings

```
┌─────────────┐    ┌────────────────┐    ┌─────────────┐    ┌──────────────┐
│   SMILES    │ →  │   ChemBERTa    │ →  │  768-dim    │ →  │  Classifier  │ → Permeable?
│   string    │    │   (FROZEN)     │    │  embedding  │    │  (TRAINED)   │    Yes / No
└─────────────┘    └────────────────┘    └─────────────┘    └──────────────┘
                   Pre-trained on                           What architecture
                   77M molecules                            should we use?
```

**The question:** What classifier architecture should we use? Let's start with the **simplest possible model** and see how far it gets us.

## Setup

In [ ]:
# Install packages
!pip install -q transformers torch pandas matplotlib scikit-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import os
import warnings
warnings.filterwarnings('ignore')

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
print("Setup complete!")

---

# Part 2: Prepare the Data

We'll use the **BBBP dataset** (Blood-Brain Barrier Penetration) containing ~2,000 molecules with experimentally measured BBB permeability.

## Load the Dataset

In [ ]:
# Data paths
DATA_DIR = "data"

# For Colab, download data if needed
if not os.path.exists(DATA_DIR):
    print("Downloading data files...")
    !mkdir -p data
    !wget -q -O data/molecules_bbbp.csv https://raw.githubusercontent.com/racousin/ai_for_sciences/main/day2/data/molecules_bbbp.csv
    print("Data downloaded!")
else:
    print(f"Data directory found: {DATA_DIR}")

In [ ]:
# Load dataset
df = pd.read_csv(os.path.join(DATA_DIR, "molecules_bbbp.csv"))

print(f"Dataset: {df.shape[0]} molecules with known BBB permeability")
print(f"\nClass distribution:")
print(df['label_name'].value_counts())

# Calculate baseline
baseline_acc = df['label'].mean()
print(f"\n→ Baseline accuracy (always predict 'permeable'): {baseline_acc:.1%}")

## Compute Molecular Embeddings with ChemBERTa

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load ChemBERTa
print("Loading ChemBERTa...")
tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
chembert = AutoModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
chembert.eval()
chembert.to(device)

print(f"ChemBERTa loaded! Embedding dimension: {chembert.config.hidden_size}")

In [ ]:
def compute_embeddings(smiles_list, batch_size=32):
    """Convert SMILES strings to embeddings using ChemBERTa."""
    embeddings = []
    
    for i in range(0, len(smiles_list), batch_size):
        batch = smiles_list[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", 
                          padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = chembert(**inputs)
            batch_emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(batch_emb)
        
        if (i // batch_size) % 20 == 0:
            print(f"  Processed {min(i+batch_size, len(smiles_list)):,}/{len(smiles_list):,}")
    
    return np.vstack(embeddings)

# Compute embeddings
print("Computing embeddings...\n")
X = compute_embeddings(df['SMILES'].tolist())
y = df['label'].values

print(f"\nEmbedding matrix: {X.shape[0]} molecules × {X.shape[1]} features")

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to PyTorch
X_train_t = torch.FloatTensor(X_train)
y_train_t = torch.FloatTensor(y_train).unsqueeze(1)
X_test_t = torch.FloatTensor(X_test)
y_test_t = torch.FloatTensor(y_test).unsqueeze(1)

# Create data loaders
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=64)

print(f"Training set: {len(X_train)} molecules")
print(f"Test set: {len(X_test)} molecules")

---

# Part 3: The Simplest Classifier

Let's start with the **simplest possible model**: a single linear layer followed by sigmoid.

```
Input (768) ──→ Linear(1) ──→ Sigmoid ──→ Probability
```

This is essentially **logistic regression** - it learns a weighted sum of the 768 embedding features.

In [ ]:
class SimpleClassifier(nn.Module):
    """The simplest possible classifier: just one linear layer."""
    
    def __init__(self, input_dim=768):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

# Create model
simple_model = SimpleClassifier(input_dim=768)
simple_model.to(device)

n_params = sum(p.numel() for p in simple_model.parameters())
print(f"Simple model: {n_params:,} parameters")
print(f"  - 768 weights (one per embedding dimension)")
print(f"  - 1 bias term")

## Training Functions

In [ ]:
def train_model(model, train_loader, test_loader, n_epochs=30, lr=0.001):
    """Train a model and return training history."""
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    history = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}
    
    for epoch in range(n_epochs):
        # Train
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Evaluate
        model.eval()
        with torch.no_grad():
            # Train metrics
            train_pred = model(X_train_t.to(device)).cpu()
            train_acc = ((train_pred > 0.5).float() == y_train_t).float().mean().item()
            
            # Test metrics
            test_pred = model(X_test_t.to(device)).cpu()
            test_loss = criterion(test_pred, y_test_t).item()
            test_acc = ((test_pred > 0.5).float() == y_test_t).float().mean().item()
        
        history['train_loss'].append(train_loss / len(train_loader))
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1:>3}: Train Acc = {train_acc:.1%}, Test Acc = {test_acc:.1%}")
    
    return history

In [ ]:
# Train the simple model
print("Training simple model (Linear + Sigmoid)...\n")
simple_history = train_model(simple_model, train_loader, test_loader, n_epochs=30)

print(f"\nFinal test accuracy: {simple_history['test_acc'][-1]:.1%}")
print(f"Baseline (always permeable): {baseline_acc:.1%}")

## Evaluate the Simple Model

In [ ]:
def evaluate_model(model, model_name="Model"):
    """Evaluate a model and display results."""
    model.eval()
    with torch.no_grad():
        y_pred_proba = model(X_test_t.to(device)).cpu().numpy().flatten()
        y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{'='*50}")
    print(f"  {model_name} - Results")
    print(f"{'='*50}")
    print(f"\nAccuracy: {acc:.1%}  (Baseline: {baseline_acc:.1%})")
    print(f"\nConfusion Matrix:")
    print(f"                  Predicted")
    print(f"                  Neg    Pos")
    print(f"  Actual Neg     {tn:4d}   {fp:4d}   (False Positives: {fp})")
    print(f"  Actual Pos     {fn:4d}   {tp:4d}   (False Negatives: {fn})")
    
    # Precision and Recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    print(f"\nPrecision: {precision:.1%}  (of predicted permeable, how many actually are)")
    print(f"Recall:    {recall:.1%}  (of actual permeable, how many we found)")
    
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 
            'fp': fp, 'fn': fn, 'y_pred_proba': y_pred_proba}

simple_results = evaluate_model(simple_model, "Simple Model (Linear + Sigmoid)")

### Question 1: Analyzing the Simple Model

Look at the results above:

1. How much better is the simple model compared to the baseline?
2. How many **false negatives** does the model make? (drugs we wrongly reject)
3. Do you think a more complex model could do better? Why?

---

# Part 4: Build a Better Model

The simple linear model has limitations:
- It can only learn **linear combinations** of features
- It cannot capture **non-linear patterns** in the data

## Exercise: Create an Improved Classifier

Your task: Build a **Multi-Layer Perceptron (MLP)** with:
- At least one hidden layer
- Non-linear activation functions (ReLU)
- Optionally: Dropout for regularization

```
Example architecture:
Input (768) → Linear(256) → ReLU → Linear(64) → ReLU → Linear(1) → Sigmoid
```

In [ ]:
class ImprovedClassifier(nn.Module):
    """TODO: Build a better classifier with hidden layers."""
    
    def __init__(self, input_dim=768):
        super().__init__()
        
        # TODO: Replace this with a better architecture!
        # Hint: Use nn.Sequential with multiple Linear layers,
        # ReLU activations, and optionally Dropout
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, 1),  # <-- Modify this!
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

# Create and check your model
improved_model = ImprovedClassifier(input_dim=768)
improved_model.to(device)

n_params = sum(p.numel() for p in improved_model.parameters())
print(f"Your model has {n_params:,} parameters")
print(f"\nArchitecture:")
print(improved_model)

In [ ]:
# Train your improved model
print("Training your improved model...\n")
improved_history = train_model(improved_model, train_loader, test_loader, n_epochs=50)

print(f"\nYour model test accuracy: {improved_history['test_acc'][-1]:.1%}")
print(f"Simple model test accuracy: {simple_history['test_acc'][-1]:.1%}")

In [ ]:
# Evaluate your improved model
improved_results = evaluate_model(improved_model, "Your Improved Model")

## Compare the Models

In [ ]:
# Side-by-side comparison
print("\n" + "="*60)
print("                    MODEL COMPARISON")
print("="*60)
print(f"{'Metric':<20} {'Simple':<15} {'Improved':<15}")
print("-"*60)
print(f"{'Accuracy':<20} {simple_results['accuracy']:<15.1%} {improved_results['accuracy']:<15.1%}")
print(f"{'Precision':<20} {simple_results['precision']:<15.1%} {improved_results['precision']:<15.1%}")
print(f"{'Recall':<20} {simple_results['recall']:<15.1%} {improved_results['recall']:<15.1%}")
print(f"{'False Positives':<20} {simple_results['fp']:<15} {improved_results['fp']:<15}")
print(f"{'False Negatives':<20} {simple_results['fn']:<15} {improved_results['fn']:<15}")
print("-"*60)
print(f"{'Baseline':<20} {baseline_acc:<15.1%}")

In [ ]:
# Plot training curves comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy
axes[0].plot(simple_history['test_acc'], label='Simple Model', linewidth=2, linestyle='--')
axes[0].plot(improved_history['test_acc'], label='Improved Model', linewidth=2)
axes[0].axhline(y=baseline_acc, color='red', linestyle=':', label=f'Baseline ({baseline_acc:.1%})', alpha=0.7)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Test Accuracy')
axes[0].set_title('Test Accuracy Comparison', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(simple_history['test_loss'], label='Simple Model', linewidth=2, linestyle='--')
axes[1].plot(improved_history['test_loss'], label='Improved Model', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Test Loss')
axes[1].set_title('Test Loss Comparison', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Question 2: Model Comparison

1. Did your improved model beat the simple model? By how much?
2. Did adding more layers help? What about dropout?
3. Is there a sign of overfitting in either model?

---

# Part 5: Understanding the Errors

In drug discovery, **not all errors are equal**.

- **False Positive**: We predict the drug crosses BBB, but it doesn't
  - *Consequence*: Wasted experiments on drugs that won't reach the brain

- **False Negative**: We predict the drug is blocked, but it actually crosses
  - *Consequence*: We might discard a life-saving drug candidate!

## Adjusting the Decision Threshold

By default, we predict "permeable" if probability > 0.5. We can change this.

In [ ]:
# Use the best model for this analysis
best_model = improved_model if improved_results['accuracy'] > simple_results['accuracy'] else simple_model
y_proba = improved_results['y_pred_proba'] if improved_results['accuracy'] > simple_results['accuracy'] else simple_results['y_pred_proba']

# Try different thresholds
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

print("Impact of Decision Threshold:")
print("=" * 70)
print(f"{'Threshold':>10} {'Accuracy':>12} {'Precision':>12} {'Recall':>12} {'False Neg':>12}")
print("-" * 70)

for thresh in thresholds:
    y_pred_t = (y_proba > thresh).astype(int)
    acc = accuracy_score(y_test, y_pred_t)
    tp = ((y_pred_t == 1) & (y_test == 1)).sum()
    fp = ((y_pred_t == 1) & (y_test == 0)).sum()
    fn = ((y_pred_t == 0) & (y_test == 1)).sum()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    marker = " ← default" if thresh == 0.5 else ""
    print(f"{thresh:>10.1f} {acc:>12.1%} {precision:>12.1%} {recall:>12.1%} {fn:>12}{marker}")

print("\n→ Lower threshold = fewer false negatives (don't miss good drugs)")
print("→ Higher threshold = fewer false positives (don't waste on bad drugs)")

### Question 3: Threshold Selection

In early-stage drug discovery for Alzheimer's, which errors are more costly?

1. If we want to **not miss any potential drug** (minimize false negatives), what threshold should we use?
2. What's the trade-off when we lower the threshold?
3. How would you choose the threshold for a real drug discovery project?

---

# Part 6: Predict Your Drug Candidate

Now let's use our model for its intended purpose: **predicting whether new molecules can cross the BBB**.

In [ ]:
def predict_bbb(smiles, model, name="Unknown"):
    """Predict BBB permeability for a molecule."""
    inputs = tokenizer(smiles, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        emb = chembert(**inputs).last_hidden_state[:, 0, :]
        prob = model(emb).cpu().item()
    
    return prob

In [ ]:
# Our candidate molecule (Donepezil - an approved Alzheimer's drug)
candidate_smiles = "COc1cc2CC(CC2cc1OC)C(=O)N1CCc2ccccc2C1"
candidate_name = "AZD-2024 (Donepezil analog)"

prob = predict_bbb(candidate_smiles, best_model, candidate_name)

print("\n" + "=" * 60)
print("          BBB PERMEABILITY PREDICTION REPORT")
print("=" * 60)
print(f"\nMolecule: {candidate_name}")
print(f"SMILES:   {candidate_smiles}")
print(f"\nPredicted probability: {prob:.1%}")
print(f"\nVerdict: {'PERMEABLE - Likely to cross BBB' if prob > 0.5 else 'IMPERMEABLE - Likely blocked'}")
print("=" * 60)

if prob > 0.5:
    print("\n→ Recommendation: Proceed with synthesis and testing.")
else:
    print("\n→ Recommendation: Consider structural modifications.")

## Test on Known Drugs

In [ ]:
# Real drugs to test
test_drugs = [
    ("Caffeine", "Cn1cnc2c1c(=O)n(c(=O)n2C)C", "SHOULD cross (CNS stimulant)"),
    ("Diazepam", "CN1C(=O)CN=C(c2ccccc2)c3cc(Cl)ccc13", "SHOULD cross (sedative)"),
    ("Ibuprofen", "CC(C)Cc1ccc(cc1)C(C)C(=O)O", "Should NOT cross (peripheral)"),
    ("Penicillin G", "CC1(C)SC2C(NC(=O)Cc3ccccc3)C(=O)N2C1C(=O)O", "Should NOT cross (antibiotic)"),
]

print("Predictions for Known Drugs:")
print("=" * 75)
print(f"{'Drug':<15} {'Probability':>12} {'Prediction':<15} {'Expected'}")
print("-" * 75)

for name, smiles, expected in test_drugs:
    prob = predict_bbb(smiles, best_model, name)
    pred = "Permeable" if prob > 0.5 else "Impermeable"
    print(f"{name:<15} {prob:>12.1%} {pred:<15} {expected}")

## Try Your Own Molecule

In [ ]:
# TODO: Replace with your own molecule!
my_smiles = "CCO"  # <-- Ethanol
my_name = "My molecule"

prob = predict_bbb(my_smiles, best_model, my_name)
print(f"\n{my_name}: {prob:.1%} probability of crossing BBB")
print(f"Prediction: {'Permeable' if prob > 0.5 else 'Impermeable'}")

---

# Summary

## What We Built

```
SMILES → ChemBERTa (frozen) → 768-dim embedding → Classifier (trained) → BBB Prediction
```

## Key Takeaways

1. **Start simple**: A linear classifier is a good baseline to beat

2. **Transfer learning works**: ChemBERTa's embeddings encode useful chemistry - we just need a small classifier on top

3. **Evaluation matters**: Accuracy isn't everything - consider which errors are costly for your application

4. **Threshold tuning**: You can trade off precision vs recall based on your needs

## This Pattern Works Everywhere

| Domain | Pre-trained Model | Classification Task |
|--------|-------------------|--------------------|
| Molecules | ChemBERTa | Toxicity, solubility, BBB |
| Proteins | ESM-2 | Function, stability |
| DNA | DNABERT-2 | Promoter detection |
| Text | SciBERT | Topic classification |

---

## Reflection Questions

1. **In your research**, what classification problems could benefit from pre-trained embeddings?

2. **What pre-trained model** exists for your data type?

3. **Which errors** would be more costly in your application?

4. **When might you need** to fine-tune the embedding model itself, instead of just training a classifier on top?

---

## Bonus: Save Your Model

In [ ]:
# Save your improved model
torch.save(improved_model.state_dict(), 'bbb_classifier.pt')
print("Model saved to 'bbb_classifier.pt'")